# Multi Agent Mentorship Matchign Algortihm Tester

In this document I will test using four speciialized AI agents that will conduct natural conversations with a "BMCC student" in attempt to build deep personality profiles that will be used to match them with their top 5 mentors.

**Why?** Traditional mentor matching has several issues surrounding the surface level matching algorithms which involve simple categorical variables inputted by the student through static forms. This can often miss the nuances in the student personality, mentor preferences, learning styles, and other actual needs to succeed within BMCC.

**Solution** - Students will use a simple UI that will them to participate in a deep 10 minute conversation with four specialized agents.
1. **Learning Style Analyzer** - Inquires about situations in which student retained information best to infer their learning styles, confidence level in their program, and where they may need support
2. **Career Objective Analyzer** - Discusses career goals and passions to pin point the targetted future centered help they may needed and professional settings the student is drawn too (like mathematic heavy, fintech, etc)
3. **Personal Circumstances Analyzer** - Deep dives into external factors that may hinder students mentorship progress, education, or confidence levels. Also picks up on communication preferences and needs beyond academic support.
4. **Personality Profile Summarizer** - Analyzes all conversation data and converts it into a readable and comprehensive profile in markdown format which includes the top 5 mentor matches.
> WIC Portal Initiative, Summer 2025, Professor Azhar

In [1]:
import requests # Handles HTTP requests 
from typing import List, Dict # Makes function parameters and returns more debuggable


In [2]:
# For beautiful markdown summaries
from IPython.display import Markdown, display

In [3]:
# Class to interact with Ollama model through API calls
class OllamaClient:
    # Constructor
    def __init__(self, base_url: str="http://localhost:11434"):
        self.base_url=base_url # Default server wher Ollama listens
        self.model = "llama3.2" # Model used in our API requests


    def generate_response(self, prompt: str, conversation_history: List[Dict] = None) -> str:
        """ Send prompt to Ollama and get an AI response """
        try:
            payload = {
                "model": self.model,
                "messages": conversation_history or [],
                "stream": False # Wait for entire response to be generated, then display
            }

            # Add latest user prompt to conversation history
            payload['messages'].append({"role":"user", "content":prompt})
            
            # Sends full conversation history as a POST request to Ollama
            response = requests.post(f"{self.base_url}/api/chat", json=payload)

            return response.json()['message']['content']

        # Return error message as string
        except Exception as e:
            return f"Error: {str(e)}"
        
        

In [4]:
# Test Ollama connection
client = OllamaClient()
test_response = client.generate_response("Say hello and nothing else")
print("Test Response:", test_response)

Test Response: Hello.


In [5]:
# Sample BMCC student data

# List containing dictionary of students
# Sample data representing diverse BMCC computing students
sample_students = [
    {
        "emplid": "S001",
        "name": "Maria Rodriguez",
        "major": "Computer Science",
        "semester": "4th semester",
        "background": "First-generation college student, works part-time at Target, wants to start own company",
        "initial_goals": "I want to work in web development and eventually start my own tech company",
        "current_challenges": "Sometimes I feel like I don't belong in tech, especially during technical interviews"
    },
    {
        "emplid": "S002", 
        "name": "Aisha Johnson",
        "major": "Computer Information Systems",
        "semester": "3rd semester",
        "background": "Returning student, former military, single mother of two kids",
        "initial_goals": "Interested in cybersecurity, want to work for government agency like NSA or DHS",
        "current_challenges": "Hard to keep up with rapidly changing technology while managing family responsibilities"
    },
    {
        "emplid": "S003",
        "name": "Jennifer Park", 
        "major": "Computer Network Technology",
        "semester": "2nd semester",
        "background": "Traditional student, good grades but lacks confidence, unsure about career direction",
        "initial_goals": "Not sure exactly what I want to do, maybe data analysis or networking?",
        "current_challenges": "Feel overwhelmed by all the different career paths, don't know which direction to focus on"
    }
]

sample_mentors = [
    {
        "mentor_id": "M001",
        "name": "Sarah Chen",
        "title": "Senior Frontend Developer",
        "company": "Spotify", 
        "years_experience": 6,
        "specialization": "React, JavaScript, UI/UX Design, Agile Development",
        "mentoring_style": "Hands-on project-based learning, regular code reviews, encouraging and patient",
        "communication_style": "Casual and friendly, prefers video calls, responds quickly to messages",
        "availability": "Evenings 6-8pm EST, Saturday mornings",
        "background": "Former BMCC student, first-generation college graduate, understands imposter syndrome",
        "preferred_mentee": "Students interested in frontend/fullstack, need confidence building"
    },
    {
        "mentor_id": "M002", 
        "name": "James Wilson",
        "title": "Cybersecurity Analyst",
        "company": "Department of Homeland Security",
        "years_experience": 8,
        "specialization": "Network Security, Incident Response, Security Clearances, Compliance", 
        "mentoring_style": "Structured guidance with clear milestones, certification-focused, goal-oriented",
        "communication_style": "Professional and detailed, prefers scheduled calls, thorough explanations",
        "availability": "Flexible schedule, prefers morning meetings 9-11am EST",
        "background": "Military veteran, career changer from non-tech field, understands adult learner challenges",
        "preferred_mentee": "Students interested in security, government work, need career direction"
    },
    {
        "mentor_id": "M003",
        "name": "Dr. Lisa Kim",
        "title": "Data Science Manager", 
        "company": "Netflix",
        "years_experience": 10,
        "specialization": "Machine Learning, Data Analytics, Python, SQL, Statistical Analysis",
        "mentoring_style": "Exploratory conversations, helps clarify career direction, research-oriented",
        "communication_style": "Patient and analytical, asks thoughtful questions, prefers in-depth discussions",
        "availability": "Weekday lunch hours 12-1pm EST, some evenings",
        "background": "PhD in Statistics, transitioned from academia to industry, mentors underrepresented groups",
        "preferred_mentee": "Students exploring data science, need help finding focus, enjoy analytical thinking"
    },
    {
        "mentor_id": "M004",
        "name": "Carlos Mendez",
        "title": "DevOps Engineer",
        "company": "Goldman Sachs",
        "years_experience": 5,
        "specialization": "Cloud Infrastructure, Docker, Kubernetes, CI/CD Pipelines, Financial Technology",
        "mentoring_style": "Problem-solving focused, teaches through real-world scenarios, emphasizes industry best practices",
        "communication_style": "Direct and practical, prefers chat/slack for quick questions, monthly video calls",
        "availability": "Weekend afternoons, occasional weekday evenings",
        "background": "Self-taught developer, community college graduate, passionate about helping non-traditional students",
        "preferred_mentee": "Students interested in backend/infrastructure, financial technology, need practical guidance"
    }
]

print(f"There are...\n{len(sample_students)} students")
print(f"{len(sample_mentors)} mentors")

There are...
3 students
4 mentors


In [6]:
class LearningStyleAgent:
    def __init__(self, ollama_client: OllamaClient):
        # Stores client so the agent can generate AI responses
        self.client = ollama_client
        # Holds list of messages between AI and the student
        self.conversation_history = []
        
    # Method to begin personalized conversation with the student
    def start_conversation(self, student_data: Dict) -> str:
        """Initiate learning style analysis conversation"""  
        system_prompt = f"""You are a learning style specialist at BMCC helping {student_data['name']}, a {student_data['major']} student in their {student_data['semester']}.
        Your goal: Understand HOW they learn best through natural conversation, not interrogation.
        
        Focus areas to explore naturally:
        - How they approach new programming concepts (trial-and-error vs structured learning)
        - When they feel most confident vs frustrated in their studies
        - Their preferred way of getting help when stuck (classmates, professors, online resources)
        - How they retain information best (doing, seeing, reading, explaining to others)
        - Their confidence patterns and where they need support

        Style: Be conversational, curious, and supportive. Ask ONE engaging question at a time. 
        Keep responses under 2 sentences. Build on their answers naturally.

        Start by asking about a recent learning experience in their program."""

        # Initialize the history with a system role message
        self.conversation_history = [{"role": "system", "content": system_prompt}]

        # More information on the student
        opening_context = f"Student background: {student_data['background']}"

        first_question = self.client.generate_response(
            f"Start a natural conversation with {student_data['name']} about their learning experiences. {opening_context}",
            self.conversation_history
        )

        # Append AI's reply to the conversation history
        self.conversation_history.append({"role": "assistant", "content": first_question})

        return first_question

    # Take student response and get next question
    def continue_conversation(self, student_response: str):
        """Continue the learning style conversation based on student's response"""
    
        # Add student's answer to history
        self.conversation_history.append({"role":"user", "content":student_response})
    
        # Generate thoughtful follow-up
        follow_up_prompt = """Based on their response, ask a thoughtful follow-up question that:
        1. Digs deeper into their learning preferences
        2. Explores how they handle challenges
        3. Understands their confidence levels
        4. Identifies their support needs
        
        Keep it conversational and natural, like a caring advisor, not an interviewer."""
    
        # Generate next question
        next_question = self.client.generate_response(follow_up_prompt, self.conversation_history)
        
        # Add next question to history
        self.conversation_history.append({"role": "assistant", "content": next_question})
    
        return next_question

    # Take conversation and summarize student learning style profile
    def analyze_learning_style(self) -> str:
        """Extract structured learning style insights from the conversation"""
    
        # Build a combined string of all conversation text except the initial system message
        conversation_text = "\n".join([
            f"{msg['role'].upper()}: {msg['content']}" 
            for msg in self.conversation_history 
            if msg['role'] != 'system'
        ])
    
        # Prompt asking for analysis
        analysis_prompt = f"""Analyze this learning style conversation and extract key insights:
    
        {conversation_text}
        
        Provide a structured analysis covering:
        
        1. **Preferred Learning Methods**: 
           - How they learn best (hands-on, visual, theoretical, social, independent)
           - Information processing style (sequential vs big-picture)
        
        2. **Challenge Response Patterns**:
           - How they handle frustration and setbacks
           - When and how they seek help
           - Persistence vs asking for help patterns
        
        3. **Confidence and Support Needs**:
           - Areas where they feel confident vs insecure
           - Types of encouragement or guidance they respond to
           - Social vs independent learning preferences
        
        4. **Communication Style**:
           - How they prefer to receive feedback
           - Formal vs casual interaction preferences
        
        Format as clear, actionable insights for mentor matching."""

        # Send prompt to model and store analysis response
        analysis = self.client.generate_response(analysis_prompt)
    
        return analysis
        
            

In [7]:
print("Testing learning styles agent")
agent1 = LearningStyleAgent(client)

Testing learning styles agent


In [8]:
Maria = sample_students[0]
first_question = agent1.start_conversation(Maria)
print(f"\n🤖 Learning Agent: {first_question}")


🤖 Learning Agent: Hi Maria, how's your day going so far? I know you had a big assignment due last night for your data structures class. How did it go? Did you find yourself working late into the night or was it one of those assignments where everything clicked?


In [9]:
Maria_learning_responses = [
    "I was trying to learn React hooks last week and got really frustrated with the documentation. I learn way better when I can see someone else code it first, then try it myself.",
    
    "Yeah, I usually try to figure things out on my own for maybe 30 minutes, but if I'm still stuck I'll ask my classmate Jose before going to the professor. It feels less intimidating.",
    
    "Definitely when I'm building something I actually want to use! Like I made a simple website for my sister's small business and everything just clicked. Way better than random practice problems.",
    
    "I guess I like when people explain the 'why' behind things, not just the 'how'. And I prefer casual conversations over formal presentations - makes me feel less stupid when I ask questions."
]

In [10]:
# Run through the conversation
for i, response in enumerate(Maria_learning_responses):
    print(f"\nMaria: {response}")
    
    if i < len(Maria_learning_responses) - 1:
        next_question = agent1.continue_conversation(response)
        print(f"\nAgent: {next_question}")
    else:
        # Just record final response without asking another question
        agent1.conversation_history.append({"role": "user", "content": response})



Maria: I was trying to learn React hooks last week and got really frustrated with the documentation. I learn way better when I can see someone else code it first, then try it myself.

Agent: React hooks can be tough to grasp at first, especially with all the nuances of state management. So, I'm curious, when you're working on a new concept or trying to understand someone else's code, do you find that you prefer watching video tutorials or coding challenges online, or is there something specific about seeing others' solutions that really helps clarify things for you?

Maria: Yeah, I usually try to figure things out on my own for maybe 30 minutes, but if I'm still stuck I'll ask my classmate Jose before going to the professor. It feels less intimidating.

Agent: That makes sense, taking the time to try things out on your own can really help solidify understanding. It's great that you have a good resource in Jose too - having a peer who's also learning alongside you can be super helpful.

In [11]:
print(f"Get learnign anaylsis")

Get learnign anaylsis


In [12]:
analysis = agent1.analyze_learning_style()
print(analysis)

**Preferred Learning Methods**

* Maria prefers hands-on learning experiences, such as coding challenges and building projects with real-world applications.
* She also values visual explanations, such as watching others' code or video tutorials, to help clarify complex concepts like React hooks.
* Informal conversations and social interactions are preferred over formal presentations or individualized instruction.

**Challenge Response Patterns**

* When faced with frustration or setbacks, Maria tries to work through the issue on her own for 30 minutes before seeking help from a classmate like Jose.
* She is more likely to ask for help when she feels less intimidated, suggesting that she values social support over independent learning.
* Maria tends to be persistent in the face of challenges, but also knows when to seek additional guidance.

**Confidence and Support Needs**

* Areas where Maria feels confident: building projects with real-world applications and explaining technical conc

In [13]:
display(Markdown(analysis))

**Preferred Learning Methods**

* Maria prefers hands-on learning experiences, such as coding challenges and building projects with real-world applications.
* She also values visual explanations, such as watching others' code or video tutorials, to help clarify complex concepts like React hooks.
* Informal conversations and social interactions are preferred over formal presentations or individualized instruction.

**Challenge Response Patterns**

* When faced with frustration or setbacks, Maria tries to work through the issue on her own for 30 minutes before seeking help from a classmate like Jose.
* She is more likely to ask for help when she feels less intimidated, suggesting that she values social support over independent learning.
* Maria tends to be persistent in the face of challenges, but also knows when to seek additional guidance.

**Confidence and Support Needs**

* Areas where Maria feels confident: building projects with real-world applications and explaining technical concepts to others.
* Areas where Maria feels insecure: theoretical or abstract concepts like state management.
* Types of encouragement that motivate Maria: seeing the practical application of new skills, such as building a website for her sister's small business.

**Communication Style**

* Maria prefers casual conversations over formal presentations or individualized instruction.
* She values feedback that provides context and explanations, rather than just generic praise or criticism.

**Actionable Insights for Mentor Matching**

1. **Pairing Suggestions**: Consider pairing Maria with a mentor who excels in hands-on learning experiences and has experience working on projects with real-world applications.
2. **Support Strategies**: Encourage Maria to seek help from her classmate Jose or other peers when feeling stuck, while also providing access to video tutorials or coding challenges that can supplement individual instruction.
3. **Confidence-Boosting Activities**: Suggest activities that allow Maria to build confidence in theoretical or abstract concepts, such as explaining technical concepts to others or participating in online communities related to computer science.
4. **Communication Preferences**: Emphasize the importance of informal conversations and social interactions in providing feedback and encouragement, while also offering flexible communication channels to accommodate Maria's preferences.

By considering these insights, mentors can tailor their support and guidance to meet Maria's unique learning needs and preferences, increasing her chances of success in computer science and entrepreneurship.

In [14]:
# Career Exploration Agent
# Help student uncover deeper motivations behind career interests
class CareerExplorationAgent:
    def __init__(self, ollama_client: OllamaClient):
        self.client = ollama_client
        self.conversation_history = []

    def start_conversation(self , student_data: Dict) -> str:
        """Begin career exploration conversation"""
        
        # Define the agent's role
        system_prompt = f"""You are a career exploration specialist helping {student_data['name']}, 
        a {student_data['major']} student. Your goal is to understand their DEEP career motivations 
        and interests, not just surface-level job titles.

        Go beyond "I want to be a software developer" to understand:
        - What kind of impact they want to make in the world
        - What type of work environment energizes vs drains them  
        - What aspects of technology genuinely excite them
        - Their long-term vision (5-10 years out)
        - Whether they're drawn to technical depth, business impact, creative problem-solving, or helping others

        Student's initial goals: "{student_data['initial_goals']}"
        
        Be genuinely curious and help them discover motivations they might not have articulated yet.
        Ask ONE thought-provoking question at a time. Keep responses conversational and under 2 sentences."""
        
        # Initialize conversation with system prompt
        self.conversation_history = [{"role": "system", "content": system_prompt}]

        # Generate first question to begin conversation on career interests
        opening_question = self.client.generate_response(
            f"Start exploring {student_data['name']}'s deeper career motivations. They mentioned: {student_data['initial_goals']}",
            self.conversation_history
        )
        # Add assistant's first question to history
        self.conversation_history.append({"role": "assistant", "content": opening_question})
        
        return opening_question

    # Create follow replies to student
    def continue_conversation(self, student_response: str) -> str:
        # Add student's response to history
        self.conversation_history.append({"role":"user", "content":student_response})

        follow_up_prompt = """Ask a follow-up question that digs deeper into their motivations:
        - If they mention specific technologies, ask what draws them to those
        - If they mention job titles, explore what kind of daily work appeals to them
        - If they mention companies, understand what work environment they're seeking
        - If they mention impact, explore what problems they want to solve
        
        Be curious about the 'why' behind their interests."""

        # Generate a deeper question
        next_question = self.client.generate_response(follow_up_prompt, self.conversation_history)

        # Add assistant's follow up question to history
        self.conversation_history.append({"role":"assistant", "content":next_question})

        return next_question

    
    def analyze_career_goals(self) -> str:
        """Take full conversation and aks the AI to analyze 
        and structure what was learned
        """
        
        # Build a combined string of all conversation text except the initial system message
        conversation_text = "\n".join([
            f"{msg['role'].upper()}: {msg['content']}"
            for msg in self.conversation_history
            if msg['role'] != 'system'
        ])
        
        # Prompt asking for analysis
        analysis_prompt = f"""Analyze this career exploration conversation:

        {conversation_text}
        
        Extract and categorize:
        
        1. **Core Motivations and Values**:
           - What truly drives their interest in technology
           - Whether they're motivated by problem-solving, creativity, impact, stability, growth, etc.
        
        2. **Work Environment Preferences**:
           - Company size and culture preferences (startup vs enterprise, collaborative vs independent)
           - Industry interests (finance, healthcare, entertainment, government, etc.)
        
        3. **Technical vs Business vs Creative Inclinations**:
           - Do they gravitate toward deep technical challenges, business applications, or creative solutions?
           - Their comfort level with math, design, communication, leadership
        
        4. **Long-term Vision and Growth Areas**:
           - Where they see themselves in 5-10 years
           - Skills or knowledge gaps they're aware of
           - Leadership vs individual contributor aspirations
        
        5. **Ideal Mentor Profile**:
           - What type of mentor would best support their specific goals
           - Industry experience, company type, or role that would be most valuable
        
        Provide actionable insights for mentor matching."""
        
        analysis = self.client.generate_response(analysis_prompt)

        return analysis
        

        

In [15]:
# test career exploration agent
print(f"Testing career agnet model")
agent2 = CareerExplorationAgent(client)
career_question = agent2.start_conversation(Maria)
print(f"Career Agent: {career_question}")

Testing career agnet model
Career Agent: So it sounds like you're drawn to the idea of building something from scratch - but what about the impact you'd like that startup to have? Would you want your business to focus on solving a specific social problem, or more on making technology accessible to underrepresented groups?


In [16]:
Maria_career_responses = [
    "I’ve always said I want to go into web development, but honestly I think what draws me to it is the idea of building tools that make everyday life easier — especially for people who are often ignored by tech.",

    "I come from a family where no one’s good with technology, so I’d love to create tools that are super intuitive — like apps for older adults, or websites that work well on low-end phones.",

    "I think I’d be happiest somewhere collaborative. I like bouncing ideas off other people and working in small teams — huge corporations feel overwhelming to me.",

    "In 5 or 10 years, I’d love to run a small dev team building accessible tech for nonprofits or schools. I want to grow as both a designer and a leader, but I know I still need to get better at backend and databases.",

    "A mentor who understands what it’s like to break into tech without connections — someone who can give both technical advice and career guidance, especially around startups or social impact work."
]

In [17]:
for i, response in enumerate(Maria_career_responses):
    print(f"Maria: {response}")

    if i < len(Maria_career_responses) - 1:
        next_question = agent2.continue_conversation(response)
        print(f"Agent: {next_question}")

    else:
        agent2.conversation_history.append({"role":"user", "content":response})


Maria: I’ve always said I want to go into web development, but honestly I think what draws me to it is the idea of building tools that make everyday life easier — especially for people who are often ignored by tech.
Agent: So you're drawn to making a positive impact on people's lives through web development... That's really interesting! When I ask myself if it's just about building cool tools or creating products that genuinely improve people's daily lives, what if the latter is more of a motivator for me? Is there a particular group of people (e.g. seniors, low-income communities) that you feel needs attention and support through your tech work?

(By the way, I'm not trying to steer you away from web development, but rather understand what drives your passion for it!)
Maria: I come from a family where no one’s good with technology, so I’d love to create tools that are super intuitive — like apps for older adults, or websites that work well on low-end phones.
Agent: Creating tools that

In [18]:
career_analysis = agent2.analyze_career_goals()
display(Markdown(career_analysis))

Based on the conversation, here are the extracted and categorized insights:

1. **Core Motivations and Values**:
	* True motivation: Creating accessible tech to improve people's daily lives, especially for those who feel left behind by technology.
	* Values: Empowerment, community, collaboration, autonomy, and making a positive impact.
2. **Work Environment Preferences**:
	* Company size preference: Startups or small teams with a collaborative culture.
	* Industry interest: Non-profit, education, or social impact work.
3. **Technical vs Business vs Creative Inclinations**:
	* Technical inclinations: Backend and databases skills are important to them for future growth.
	* Business inclinations: Not explicitly stated, but they're interested in creating products that improve people's lives.
	* Creative inclinations: Designing intuitive tools and apps for underrepresented groups.
4. **Long-term Vision and Growth Areas**:
	* Long-term vision: Running a small dev team building accessible tech for non-profits or schools.
	* Growth areas: Developing backend skills, leadership experience, and growing as both a designer and leader.
5. **Ideal Mentor Profile**:
	* Industry experience: Social impact work or startups in the tech industry.
	* Company type: Non-profit or education-focused organizations would be ideal.
	* Role: A mentor who provides both technical advice and career guidance, with a focus on social impact and startup experiences.

Actionable insights for mentor matching:

1. **Mentorship style**: Look for mentors who are experienced in social impact work, startups, or non-profit organizations. They should have a collaborative mindset and be able to provide both technical advice and career guidance.
2. **Industry connections**: Establish relationships with mentors who can introduce you to industry leaders, networks, or resources that align with your values and goals.
3. **Skill development**: Focus on developing backend skills and exploring leadership opportunities within non-profit or education-focused organizations.
4. **Community engagement**: Engage with online communities, attend events, and participate in hackathons related to social impact and accessible tech to network with like-minded individuals.
5. **Personalized guidance**: Seek mentors who can provide personalized guidance on navigating the industry, overcoming challenges, and building a successful career in social impact work.

By considering these insights, you can find a mentor who can support your specific goals and help you grow as a designer and leader in the tech industry.

In [19]:
# Agent to discuss real-life challenges
class PersonalCircumstancesAgent:
    # Constructor
    def __init__(self, ollama_client: OllamaClient):
        self.client = ollama_client
        self.conversation_history = []

    def start_conversation(self, student_data: Dict) -> str:

        """Begin personal circumstances exploration"""
        
        # Tell agent what role to take
        system_prompt = f"""You are a holistic student support specialist helping {student_data['name']}.
        Your goal is to understand the REAL-WORLD factors that affect their academic success and mentorship needs.

        Explore sensitively and naturally:
        - Work/family/commute schedules and constraints
        - Financial pressures or time limitations
        - Social support systems (or lack thereof)
        - Communication preferences and comfort levels
        - Stress factors and coping strategies
        - Non-academic support needs (confidence building, networking, career navigation)

        Background context: "{student_data['background']}"
        Current challenges: "{student_data['current_challenges']}"
        
        Be empathetic and non-judgmental. These students often juggle multiple responsibilities.
        Ask ONE supportive question at a time, keeping responses brief and caring."""

        # Initialize conversation with system prompt
        self.conversation_history = [{"role": "system", "content": system_prompt}]
        
        # Generate first question
        opening_question = self.client.generate_response(
            f"Start a supportive conversation with {student_data['name']} about their current situation and needs beyond academics.",
            self.conversation_history
        )

        # Add first question to history
        self.conversation_history.append({"role": "assistant", "content": opening_question})
        
        return opening_question

    def continue_conversation(self, student_response: str) -> str:
        """Continue exploring personal circumstances"""

        # Append student response to history
        self.conversation_history.append({"role": "user", "content": student_response})

        # Create follow up question prompt
        follow_up_prompt = """Ask a follow-up that explores:
        - Time constraints and scheduling needs
        - Support systems and communication preferences  
        - Stress factors and coping strategies
        - Confidence levels and imposter syndrome
        - Practical barriers to success
        
        Be supportive and focus on understanding their whole situation."""

        # Generate follow up question
        next_question = self.client.generate_response(follow_up_prompt, self.conversation_history)

        # Append question to history
        self.conversation_history.append({"role": "assistant", "content": next_question})
        
        return next_question

    def analyze_circumstances(self) -> str:
        """Extract insights about student's life circumstances"""
        
        # Build a combined string of all conversation text except the initial system message
        conversation_text = "\n".join([
            f"{msg['role'].upper()}: {msg['content']}" 
            for msg in self.conversation_history 
            if msg['role'] != 'system'
        ])

        # Prompt asking for analysis
        analysis_prompt = f"""Analyze this personal circumstances conversation:

        {conversation_text}
        
        Identify key factors:
        
        1. **Time and Availability Constraints**:
           - Work schedule, family obligations, commute time
           - Best times for mentorship meetings
           - Flexibility vs structured scheduling needs
        
        2. **Communication Preferences**:
           - Preferred communication methods (text, calls, video, email)
           - Formal vs informal interaction style
           - Response time expectations and availability
        
        3. **Support Needs Beyond Academics**:
           - Confidence building and imposter syndrome issues
           - Professional networking and industry connections
           - Career navigation and decision-making support
           - Work-life balance strategies
        
        4. **Stress Factors and Barriers**:
           - Financial pressures affecting study time
           - Family responsibilities and competing priorities
           - Social isolation or lack of peer support
           - Technology access or workspace limitations
        
        5. **Ideal Mentorship Structure**:
           - Meeting frequency and format that works for their lifestyle
           - Type of support that would be most valuable
           - Mentor characteristics that would build trust and rapport

        Focus on practical insights for creating successful mentorship matches."""

        # Generate analysis
        analysis = self.client.generate_response(analysis_prompt)
        
        return analysis

In [20]:
print("Testing personal circumstances agent")
agent3 = PersonalCircumstancesAgent(client)
personal_question = agent3.start_conversation(Maria)
print(f"Circumstances Agent: {personal_question}")

Testing personal circumstances agent
Circumstances Agent: Maria, I'm so glad we're taking the time to chat today. I can sense that you're feeling a bit uncertain about your place in tech, which is totally normal. Before we dive into that, can you tell me a little more about what's been going on during those technical interviews? What specifically makes you feel like you don't belong or struggle to keep up?

(Note: I'm actively listening and nodding, making sure Maria feels comfortable sharing her thoughts and concerns.)


In [21]:
Maria_circumstances_responses = [
    "I work part-time at Target and usually do evening shifts, so I try to fit my classes and homework in the mornings or early afternoons. It gets exhausting though, especially during midterms.",
    
    "My commute isn't too bad, but I still lose about 1.5 hours total each day on the train. I usually try to study on my phone or just review notes, but it's hard to focus.",
    
    "My family is supportive emotionally, but they don’t really understand what I’m doing in tech, so I often feel like I’m navigating everything alone.",
    
    "Financially, things are tight. I'm covering my own expenses, and sometimes I skip buying textbooks or avoid taking extra classes just to make things work.",
    
    "Honestly, I struggle a lot with imposter syndrome. When I’m in a room with students who already know so much, I feel like I don’t belong — even though I’m trying my best."
]

In [22]:
# Continue conversation
for i, response in enumerate(Maria_circumstances_responses):
    print(f"\nMaria: {response}")
    
    if i < len(Maria_circumstances_responses) - 1:
        next_question = agent3.continue_conversation(response)
        print(f"\nAgent: {next_question}")
        
    else:
        # Append final user input without asking another question
        agent3.conversation_history.append({"role": "user", "content": response})



Maria: I work part-time at Target and usually do evening shifts, so I try to fit my classes and homework in the mornings or early afternoons. It gets exhausting though, especially during midterms.

Agent: It sounds like you're juggling a lot, especially with those evening shifts at Target. It's no wonder you're feeling exhausted, especially during midterms.

Can I ask, how do you currently manage your time to balance work, classes, and other responsibilities? Do you have any tools or systems in place that help you stay organized, or is it more of a "winging it" approach?

(Also, I want to acknowledge that it's amazing you're prioritizing your education and part-time job at the same time. That takes a lot of effort and dedication.)

(Note: I'm actively listening and nodding, making sure Maria feels comfortable sharing her thoughts and concerns.)

Maria: My commute isn't too bad, but I still lose about 1.5 hours total each day on the train. I usually try to study on my phone or just rev

In [23]:
circumstances_summary = agent3.analyze_circumstances()
display(Markdown(circumstances_summary))

Based on the conversation, here are some key factors that can inform the creation of a supportive mentorship structure:

1. **Time and Availability Constraints**:
	* Best times for meetings: Maria's work schedule, family obligations, and commute time make it challenging to find dedicated blocks of time.
	* Flexibility vs structured scheduling needs: Maria prefers flexible meeting arrangements, but also appreciates having some structure in place.
2. **Communication Preferences**:
	* Preferred communication methods: Text, calls, or video are all acceptable, but email is not preferred for formal interactions.
	* Formal vs informal interaction style: Maria values a more informal approach, as she feels it helps build rapport and trust with her mentor.
3. **Support Needs Beyond Academics**:
	* Confidence building and imposter syndrome issues: Maria struggles with feelings of inadequacy in the tech industry, so support for building confidence would be valuable.
	* Professional networking and industry connections: Maria wants to expand her network and gain more industry insights, but feels uncertain about how to approach this.
4. **Stress Factors and Barriers**:
	* Financial pressures affecting study time: Maria's financial situation is stressful, and she often has to make tough choices about how to allocate her resources.
	* Family responsibilities and competing priorities: Maria values her family's emotional support but feels like she's navigating everything alone in terms of career decisions.
	* Social isolation or lack of peer support: Maria might benefit from connecting with peers who share similar interests and challenges.
5. **Ideal Mentorship Structure**:
	* Meeting frequency and format that works for their lifestyle: Flexible, regular meetings that accommodate Maria's work schedule would be ideal.
	* Type of support that would be most valuable: Maria wants guidance on navigating the tech industry, building confidence, and managing imposter syndrome.
	* Mentor characteristics that would build trust and rapport: A mentor who is understanding, approachable, and willing to listen and offer constructive feedback would help Maria feel more at ease.

To create a successful mentorship match for Maria, consider the following:

* Schedule regular meetings that accommodate her work schedule and lifestyle.
* Use flexible communication methods that prioritize Maria's preferences (e.g., text or video calls instead of email).
* Offer support for building confidence, navigating the tech industry, and managing imposter syndrome.
* Emphasize the importance of family emotional support while also encouraging peer connections to expand Maria's network.
* Prioritize a mentorship structure that balances flexibility with some level of structure and accountability.

By understanding these key factors, you can create a tailored mentorship plan that addresses Maria's unique needs and supports her growth in the tech industry.

In [28]:
# Combine all classes into one profile
class ProfileAnalyzerAgent:
    """Generate student profile using the results from the different
    conversational agents"""

    def __init__(self, ollama_client: OllamaClient):
        self.client = ollama_client

    # Synthesize student info and conversation history into profile
    def create_profile(self, student_data: Dict, all_analysis: Dict[str, str]) -> str:
        """
            Synthesize all conversation data into comprehensive student profile

            Provides llm with student data, learning style, career goals, 
            and personal challengesand converts them into sections with a final
            ideal mentor traits analysis
        """
        
        synthesis_prompt = f"""Create a comprehensive mentorship profile for {student_data['name']} based on these AI agent analyses:

        STUDENT BACKGROUND:
        {student_data}
        
        LEARNING STYLE ANALYSIS:
        {all_analysis['learning_style']}
        
        CAREER OBJECTIVES ANALYSIS:  
        {all_analysis['career_goals']}
        
        PERSONAL CIRCUMSTANCES ANALYSIS:
        {all_analysis['personal_circumstances']}
        
        Create a unified profile that includes:
        
        ## Student Profile: {student_data['name']}
        
        ### Career Motivations & Goals
        [Synthesize their deep motivations, interests, and career direction]
        
        ### Learning Style & Preferences  
        [How they learn best, handle challenges, and prefer to receive support]
        
        ### Communication & Interaction Style
        [Preferred communication methods, formality level, meeting preferences]
        
        ### Personal Circumstances & Support Needs
        [Time constraints, life situation, specific support needs beyond academics]
        
        ### Ideal Mentor Characteristics
        [What type of mentor would best serve their specific needs and goals]
        
        Make this profile actionable for matching with mentors. Focus on compatibility factors that traditional forms would never capture."""

        # Generate llm profile
        profile = self.client.generate_response(synthesis_prompt)

        return profile

# Rank mentors based on how well they match a students profile
class MatchingEngine:
    def __init__(self, ollama_client: OllamaClient):
        self.client = ollama_client

    # Match student based on profile and available mentor dictionaries
    def find_best_matches(self, student_profile: str, mentors: List[Dict]) -> str:
        
        # Join all mentor summaries into a formatted string
        mentor_profiles = "\n\n".join([
            f"**{mentor['name']}** ({mentor['mentor_id']})\n"
            f"Role: {mentor['title']} at {mentor['company']}\n"
            f"Specialization: {mentor['specialization']}\n"
            f"Mentoring Style: {mentor['mentoring_style']}\n"
            f"Communication: {mentor['communication_style']}\n"
            f"Availability: {mentor['availability']}\n"
            f"Background: {mentor['background']}\n"
            f"Preferred Mentee: {mentor['preferred_mentee']}"
            for mentor in mentors
        ])

        # Tells the llm how to rank mentors by considering
        # Career alignment, communication styles, availability, etc
        matching_prompt = f"""You are an expert mentorship coordinator. Analyze this student profile and rank the mentors by compatibility.

        STUDENT PROFILE:
        {student_profile}
        
        AVAILABLE MENTORS:
        {mentor_profiles}
        
        For each mentor, provide:
        1. **Compatibility Score** (0-100)
        2. **Why This Match Works** (specific alignment factors)  
        3. **Potential Challenges** (areas that might not align perfectly)
        4. **Recommended Meeting Structure** (frequency, format, focus areas)
        
        Rank the mentors from best to worst match. Consider:
        - Career goal alignment and industry relevance
        - Learning style compatibility with mentoring approach
        - Communication style match
        - Schedule and availability fit
        - Background and experience relevance
        - Personality and support needs alignment
        
        Format as a clear ranking with detailed explanations."""

        # Generate match analysis
        matches = self.client.generate_response(matching_prompt)

        return matches



In [29]:
# Test comprehensive profile generation

# Initialize agents
test_profile = ProfileAnalyzerAgent(client)
matching_alg = MatchingEngine(client)

# Using Maria for the profile
student = sample_students[0]

# Agent analyses
analyses = {
    'learning_style': analysis,
    'career_goals': career_analysis,
    'personal_circumstances': circumstances_summary
}

# Generate mentorship profile
student_profile = test_profile.create_profile(student, analyses)
display(Markdown(student_profile))

print("Top 5 mentor matches")
matches = matching_alg.find_best_matches(student, sample_mentors)
display(Markdown(matches))
    

## Maria Rodriguez's Comprehensive Mentorship Profile

### Student Background

Maria is a 4th-semester Computer Science major, first-generation college student, and part-time Target employee who aspires to start her own tech company. She aims to work in web development and create products that improve people's daily lives.

### Learning Style Analysis

*   **Preferred Learning Methods**:
    *   Hands-on learning experiences
    *   Visual explanations (video tutorials, watching others' code)
    *   Informal conversations and social interactions
*   **Challenge Response Patterns**:
    *   Works alone for 30 minutes before seeking help from a classmate like Jose when faced with frustration or setbacks.
    *   Values social support over independent learning, but tends to be persistent in the face of challenges.
*   **Confidence and Support Needs**:
    *   Confident areas: building projects with real-world applications and explaining technical concepts to others.
    *   Insecure areas: theoretical or abstract concepts like state management.
    *   Types of encouragement that motivate her: seeing practical application of new skills, such as building a website for her sister's small business.

### Career Objectives Analysis

*   **Core Motivations & Values**:
    *   True motivation: Creating accessible tech to improve people's daily lives, especially those left behind by technology.
    *   Values: Empowerment, community, collaboration, autonomy, and making a positive impact.
*   **Work Environment Preferences**:
    *   Company size preference: Startups or small teams with a collaborative culture.
    *   Industry interest: Non-profit, education, or social impact work.
*   **Technical vs Business vs Creative Inclinations**:
    *   Technical inclinations: Backend and databases skills for future growth.
    *   Business inclinations: Creating products that improve people's lives (not explicitly stated but interested in).
    *   Creative inclinations: Designing intuitive tools and apps for underrepresented groups.
*   **Long-term Vision & Growth Areas**:
    *   Long-term vision: Running a small dev team building accessible tech for non-profits or schools.
    *   Growth areas: Developing backend skills, leadership experience, and growing as both a designer and leader.
*   **Ideal Mentor Profile**:
    *   Industry experience: Social impact work or startups in the tech industry.
    *   Company type: Non-profit or education-focused organizations ideal.
    *   Role: A mentor who provides both technical advice and career guidance with a focus on social impact and startup experiences.

### Personal Circumstances Analysis

*   **Time & Availability Constraints**:
    *   Best times for meetings: Maria's work schedule, family obligations, and commute time make it challenging to find dedicated blocks of time.
    *   Flexibility vs structured scheduling needs: Maria prefers flexible meeting arrangements but appreciates having some structure in place.
*   **Communication Preferences**:
    *   Preferred communication methods: Text, calls, or video are all acceptable but email is not preferred for formal interactions.
    *   Formal vs informal interaction style: Maria values an informal approach as it helps build rapport and trust with her mentor.
*   **Support Needs Beyond Academics**:
    *   Confidence building and imposter syndrome issues: Maria struggles with feelings of inadequacy in the tech industry, so support for building confidence would be valuable.
    *   Professional networking and industry connections: Maria wants to expand her network and gain more industry insights but feels uncertain about how to approach this.
*   **Stress Factors & Barriers**:
    *   Financial pressures affecting study time: Maria's financial situation is stressful, often forcing her to make tough resource allocation choices.
    *   Family responsibilities and competing priorities: Maria values family emotional support but feels she navigates everything alone in terms of career decisions.
    *   Social isolation or lack of peer support: Maria might benefit from connecting with peers who share similar interests and challenges.

### Ideal Mentorship Structure

*   Meeting frequency and format that works for their lifestyle: Flexible, regular meetings accommodating Maria's work schedule would be ideal.
*   Type of support that would be most valuable: Guidance on navigating the tech industry, building confidence, and managing imposter syndrome.
*   Mentor characteristics that build trust and rapport: A mentor who is understanding, approachable, and willing to listen and offer constructive feedback.

### Unified Profile for Mentor Matching

| **Category** | **Preferred/Recommended** |
| --- | --- |
| **Communication Style** | Informal conversations preferred over formal presentations or individualized instruction. |
| **Meeting Format** | Flexible meeting arrangements accommodating Maria's work schedule ideal. |
| **Support Needs** | Confidence building, industry connections, and navigating imposter syndrome important. |
| **Mentor Characteristics** | Understanding, approachable, and willing to listen with constructive feedback. |
| **Industry Experience** | Social impact work or startups in the tech industry ideal. |
| **Company Type** | Non-profit or education-focused organizations preferred. |

By considering these insights and preferences, mentors can tailor their support and guidance to meet Maria's unique needs and increase her chances of success in computer science and entrepreneurship.

This unified profile combines all aspects of Maria's personality, learning style, career aspirations, and personal circumstances into a comprehensive framework for mentor matching. It highlights the importance of flexibility, social support, confidence building, industry connections, and a mentor with the right characteristics to effectively guide Maria towards her goals.

Top 5 mentor matches


**Ranking of Mentors from Best to Worst Match**

1. **Sarah Chen (M001)**
 * Compatibility Score: 92
 * Why This Match Works:
 + Aligns well with Maria's career goals in web development and startup ambitions.
 + Both have hands-on, project-based learning experiences, making this a natural fit for Maria's needs.
 + Sarah's casual and friendly communication style matches Maria's preferences.
 + Their background as first-generation college students also creates a strong bond between them.
 * Potential Challenges: James Wilson's structured guidance might clash with Sarah's more flexible approach.
 + Recommended Meeting Structure: Bi-weekly video calls, focusing on hands-on projects and code reviews.

2. **Dr. Lisa Kim (M003)**
 * Compatibility Score: 88
 * Why This Match Works:
 + Maria's interest in data science aligns perfectly with Dr. Kim's expertise.
 + The exploration of career direction through conversational learning resonates with Maria's need for guidance.
 + Both value analytical thinking, making their communication style a strong match.
 * Potential Challenges: Maria might find the structured approach too rigid or slow-paced.
 + Recommended Meeting Structure: Weekly video calls, focusing on in-depth discussions and research-oriented projects.

3. **Carlos Mendez (M004)**
 * Compatibility Score: 80
 * Why This Match Works:
 + Carlos's practical guidance aligns with Maria's desire for hands-on experience in backend/infrastructure development.
 + His self-taught developer background provides a valuable perspective on non-traditional learning paths.
 + Direct communication and video calls suit Maria's needs for quick questions and guidance.
 * Potential Challenges: Carlos's focus on industry best practices might not align perfectly with Maria's ambition to start her own company.
 + Recommended Meeting Structure: Monthly video calls, focusing on practical problem-solving and code reviews.

4. **James Wilson (M002)**
 * Compatibility Score: 68
 * Why This Match Works:
 + James understands adult learner challenges, which benefits Maria as a part-time student.
 + The cybersecurity field aligns with Maria's interests in security, although the specific focus might be too narrow for her startup ambitions.
 + Structured guidance and certification-focused mentoring suits Maria's need for direction.
 * Potential Challenges: James's structured approach can be too rigid or slow-paced for Maria's needs.
 + Recommended Meeting Structure: Bi-weekly video calls, focusing on structured learning milestones and career planning.

5. **Sarah Chen (M001) is ranked first due to her alignment with Maria's goals, hands-on learning style, and casual communication approach. Dr. Lisa Kim comes in second due to their shared interest in data science and exploration of career direction. Carlos Mendez ranks third for his focus on backend/infrastructure development and practical guidance. James Wilson rounds out the list as a good match but with some compatibility concerns regarding structure and pace.

Please note that individual preferences might vary, and it's essential to consider each student's unique needs when selecting mentors. This ranking aims to provide a general assessment of the mentorship potential based on the provided information.

In [32]:
def complete_tester(student_data: Dict, analysis: str, career_analysis: str, circumstances_summary: str):
    """
        Run entire multi-agent matching process for one student 
        Parameters:
            student_data := contains dictionary data of a singular student
            analysis := analysis generated by learning style agent
            career_analysis := analysis generated by career exploration agent
            circumstances_summary := analysis generated by personal circumstances agent
            
    """
    # Initialize agents
    test_profile = ProfileAnalyzerAgent(client)
    matching_alg = MatchingEngine(client)
    
    # Agent analyses
    analyses = {
        'learning_style': analysis,
        'career_goals': career_analysis,
        'personal_circumstances': circumstances_summary
    }
    
    # Generate mentorship profile
    student_profile = test_profile.create_profile(student_data, analyses)
    display(Markdown(student_profile))
    
    print("Top 5 mentor matches")
    matches = matching_alg.find_best_matches(student_data, sample_mentors)
    display(Markdown(matches))

In [33]:
complete_tester(sample_students[0], analysis, career_analysis, circumstances_summary)

Here is a comprehensive mentorship profile for Maria Rodriguez:

**Student Profile: Maria Rodriguez**

### Career Motivations & Goals

Maria's primary motivation is to create accessible tech that improves people's daily lives, particularly for those who feel left behind by technology. She values empowerment, community, collaboration, autonomy, and making a positive impact. Her long-term vision is to run a small dev team building accessible tech for non-profits or schools.

### Learning Style & Preferences

* Hands-on learning experiences: Maria prefers coding challenges and building projects with real-world applications.
* Visual explanations: She values watching others' code, video tutorials, or informal conversations to clarify complex concepts like React hooks.
* Informal interactions: Maria feels more comfortable in informal settings than formal presentations or individualized instruction.

### Communication & Interaction Style

* Preferred communication methods: Text, calls, or video; email is not preferred for formal interactions.
* Formality level: Maria values an informal approach, feeling it helps build rapport and trust with her mentor.
* Meeting preferences: Flexible, regular meetings that accommodate her work schedule would be ideal.

### Personal Circumstances & Support Needs

* Time constraints: Best times for meetings are when she has dedicated blocks of time; flexible scheduling is necessary due to work and family obligations.
* Life situation: Maria values family emotional support while navigating career decisions independently.
* Specific support needs:
	+ Confidence building: Help her overcome feelings of inadequacy in the tech industry.
	+ Professional networking: Expand her network and gain more industry insights without feeling uncertain about how to approach this.

### Ideal Mentor Characteristics

* Industry experience: Social impact work or startups in the tech industry.
* Company type: Non-profit or education-focused organizations are ideal.
* Role: A mentor who provides both technical advice and career guidance, with a focus on social impact and startup experiences.

**Actionable Insights for Mentor Matching**

1. **Pairing Suggestions**: Consider pairing Maria with mentors who excel in hands-on learning experiences and have experience working on projects with real-world applications.
2. **Support Strategies**: Encourage Maria to seek help from her classmate Jose or other peers when feeling stuck, supplementing individual instruction with video tutorials or coding challenges.
3. **Confidence-Boosting Activities**: Suggest activities that allow Maria to build confidence in theoretical or abstract concepts, such as explaining technical concepts to others or participating in online communities related to computer science.
4. **Communication Preferences**: Emphasize the importance of informal conversations and social interactions in providing feedback and encouragement while offering flexible communication channels to accommodate Maria's preferences.

**Personality Traits & Strengths**

* Empathetic: Understanding that technology should improve people's lives.
* Resourceful: Adapting to challenges by working through issues alone before seeking help.
* Persistent: Tackling frustrations head-on with a willingness to learn and improve.
* Confident in practical applications, but uncertain about theoretical concepts.

**Common Challenges & Opportunities**

* Overcoming imposter syndrome and building confidence in the tech industry.
* Expanding her professional network while maintaining independence.
* Finding opportunities for growth within non-profit or education-focused organizations.

By understanding Maria's unique needs, learning style, and preferences, mentors can tailor their support to help her achieve her career goals in social impact work.

Top 5 mentor matches


Based on the provided student profile and mentor information, I've analyzed each mentor's compatibility with Maria Rodriguez. Here's the ranking from best to worst match:

**1. Sarah Chen (M001) - Compatibility Score: 92**

Why This Match Works:
- Career goal alignment: Maria wants to work in web development, and Sarah is a senior frontend developer.
- Learning style compatibility: Sarah's hands-on project-based learning approach aligns with Maria's need for confidence building.
- Communication style match: Their casual and friendly communication styles will facilitate open discussions.
- Schedule and availability fit: Sarah's evening availability (6-8pm EST) and Saturday mornings overlap with Maria's busy schedule.

Potential Challenges:
- Limited industry relevance in the context of Maria's broader career goals, such as starting her own company.

Recommended Meeting Structure:
- Bi-weekly video calls to discuss projects and progress
- Regular code reviews to provide feedback on Maria's web development work
- Quarterly "office hours" for Maria to receive guidance on project planning and direction

**2. Carlos Mendez (M004) - Compatibility Score: 85**

Why This Match Works:
- Career goal alignment: Maria is interested in web development, which aligns with Carlos' focus on backend/infrastructure.
- Learning style compatibility: Carlos' problem-solving approach will help Maria tackle practical challenges.
- Communication style match: Their direct and practical communication styles will facilitate efficient discussions.

Potential Challenges:
- Limited industry relevance in the context of Maria's broader career goals, such as starting her own company.
- Potential mismatch in learning pace due to Carlos' self-taught background.

Recommended Meeting Structure:
- Weekly chat sessions for quick questions and guidance
- Monthly video calls to discuss project progress and provide feedback
- Quarterly "mentoring sprints" where Carlos provides in-depth guidance on specific technical topics

**3. Dr. Lisa Kim (M003) - Compatibility Score: 78**

Why This Match Works:
- Career goal alignment: Maria is interested in exploring data science, which aligns with Dr. Kim's specialization.
- Learning style compatibility: Dr. Kim's exploratory conversations will help Maria clarify her career direction.

Potential Challenges:
- Limited industry relevance in the context of Maria's broader career goals, such as starting her own company.
- Potential mismatch in communication style due to Dr. Kim's more analytical and patient approach.

Recommended Meeting Structure:
- Bi-weekly video calls to discuss project progress and provide feedback
- Quarterly "research seminars" where Dr. Kim shares insights on data science topics
- Monthly "office hours" for Maria to receive guidance on research-oriented projects

**4. James Wilson (M002) - Compatibility Score: 65**

Why This Match Works:
- Career goal alignment: Maria is interested in cybersecurity, which aligns with James' specialization.
- Background and experience relevance: James' background as a military veteran provides valuable insight for adult learners.

Potential Challenges:
- Limited learning style compatibility due to James' structured guidance approach, which may not suit Maria's need for hands-on project-based learning.
- Potential mismatch in communication style due to James' more professional and detailed approach.

Recommended Meeting Structure:
- Bi-weekly video calls to discuss career direction and provide guidance
- Quarterly "career development workshops" where James shares insights on cybersecurity best practices
- Monthly "office hours" for Maria to receive guidance on career-related topics

Overall, Sarah Chen's compatibility score is the highest due to her alignment with Maria's web development goals and learning style. Carlos Mendez and Dr. Lisa Kim are close second and third, respectively, despite some potential challenges in their mentorship approach. James Wilson's compatibility score is lower due to potential mismatches in communication style and learning pace.